In [1]:
import numpy as np
import xarray
import pandas as pd

In [2]:
GFRAC = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_NEW.nc", engine="netcdf4")
GAREA = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")
GLCT = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")
GLCT_1st = xarray.open_dataset('D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc', engine="netcdf4")

array(['trof', 'trof', 'trof', 'agri', 'agri', 'agri'], dtype=object)

In [3]:
glct_array = np.where(GLCT['GLCT_trans'].isel(time=slice(0,11)).to_numpy() == "agri", 1, 0)

In [22]:
# asumsi glct_array sudah didefinisikan sebelumnya
area_pasture_thn1st = np.zeros((1, 2160, 4320), dtype='float32')

for n in range(1, 11):
    # jika tahun awal == 0 (bukan agri) dan tahun depan == 1 (agri)
    area_pasture_thn1st[0] += np.where(GFRAC['GFRAC_new'].isel(time=n, NGFBFC=0).to_numpy() > 0,
                                        np.where(glct_array[n-1] == 0, GFRAC['GFRAC_new'].isel(time=n, NGFBFC=0).to_numpy(), 0), 0)

In [31]:
area_pasture_thn1st_netcdf = xarray.Dataset({
    "area_pasture":(["latitude", "longitude"], area_pasture_thn1st[0])
},coords={
        "latitude": GFRAC.coords["latitude"].to_numpy(),
        "longitude": GFRAC.coords["longitude"].to_numpy()
})

In [33]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]
n = 1

area_pasture_thn1st_netcdf['area_pasture'].sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

array(0.6818724, dtype=float32)

In [30]:
# asumsi glct_array sudah didefinisikan sebelumnya
area_pasture_thn1end = np.zeros((1, 2160, 4320), dtype='float32')
 
area_pasture_thn1end[0] = np.where(GFRAC['GFRAC_new'].isel(time=11, NGFBFC=0).to_numpy() > 0, GFRAC['GFRAC_new'].isel(time=11, NGFBFC=0).to_numpy(), 0)

In [44]:
# if pasture tahun pertama true dan tahun terakhir true:
#  ambil nilai tahun pertama sebagai tahun pertama dan ambil nilai terakhir sebagai tahun terakhir
result_kondisi_a_tahunawal = np.where(area_pasture_thn1st[0] > 0, np.where(area_pasture_thn1end[0] > 0, area_pasture_thn1st[0], 0), 0).sum()
result_kondisi_a_tahunakhir = np.where(area_pasture_thn1st[0] > 0, np.where(area_pasture_thn1end[0] > 0, area_pasture_thn1end[0], 0), 0).sum()

In [45]:
result_kondisi_b_tahunawal = np.where(area_pasture_thn1st[0] > 0, np.where(area_pasture_thn1end[0] == 0, area_pasture_thn1st[0], 0), 0).sum()
result_kondisi_b_tahunakhir = np.where(area_pasture_thn1st[0] > 0, np.where(area_pasture_thn1end[0] == 0, area_pasture_thn1end[0], 0), 0).sum()

In [48]:
# asumsi glct_array sudah didefinisikan sebelumnya
area_pasture_dari_awal = np.zeros((1, 2160, 4320), dtype='float32')

area_pasture_dari_awal[0] = np.where(GFRAC['GFRAC_new'].isel(time=0, NGFBFC=0).to_numpy() > 0, GFRAC['GFRAC_new'].isel(time=0, NGFBFC=0).to_numpy(), 0)

In [49]:
result_kondisi_c_tahunawal = np.where(area_pasture_dari_awal[0] > 0, np.where(area_pasture_thn1end[0] > 0, area_pasture_thn1st[0], 0), 0).sum()
result_kondisi_c_tahunakhir = np.where(area_pasture_dari_awal[0] > 0, np.where(area_pasture_thn1end[0] > 0, area_pasture_thn1end[0], 0), 0).sum()

In [53]:
df = pd.DataFrame({
    "Tahun Awal": [result_kondisi_a_tahunawal, result_kondisi_b_tahunawal, result_kondisi_c_tahunawal],
    "Tahun Akhir": [result_kondisi_a_tahunakhir, result_kondisi_b_tahunakhir, result_kondisi_c_tahunakhir]
}, index=["Kondisi A", "Kondisi B", "Kondisi C"])
df

,Tahun Awal,Tahun Akhir
Kondisi A,1.651735e+06,208988.078125
Kondisi B,3.679683e+03,0.000000
Kondisi C,1.604261e+06,415406.250000
